In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

if os.getenv("GOOGLE_API_KEY") is None:
    raise ValueError("GOOGLE_API_KEY is not set in the environment variables.")
print("GOOGLE_API_KEY is set.")

GOOGLE_API_KEY is set.


In [4]:

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    # google_api_key=os.getenv("GEMINI_API_KEY"),
    model="gemini-3-flash-preview",
    temperature=0
)
# response = llm.invoke("what is rag application explain to me.")
# print(response.content)

c:\Users\rohan\Documents\Rohan\RohanDabasAll\rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm.invoke("what is langchain used for?")

AIMessage(content=[{'type': 'text', 'text': 'At its core, **LangChain** is an open-source framework designed to help developers build applications powered by Large Language Models (LLMs) like GPT-4, Claude, or Llama.\n\nWhile LLMs are powerful on their own, they have limitations: they are "stateless" (they don\'t remember past conversations naturally), they have a cutoff date for their knowledge, and they can\'t interact with your private data or external tools out of the box. **LangChain acts as the "glue"** that connects these models to other sources of data and computation.\n\nHere is a breakdown of what LangChain is used for:\n\n---\n\n### 1. Retrieval Augmented Generation (RAG)\nThis is the most common use case. LangChain allows you to connect an LLM to your own data (PDFs, databases, emails, or websites).\n*   **How it works:** When a user asks a question, LangChain searches your documents for the relevant information and feeds that specific info to the LLM to generate an accurat

In [7]:
## **RAG IMPLEMENTATION WITH OWN TEXT DATA**


In [ ]:
## step 1: preparing document for your text

In [5]:
from langchain_core.documents import Document
#your text data
text_data = """Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]

High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); 
generative and creative tools (e.g., language models and AI art); 
and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: 
"A lot of cutting edge AI has filtered into general applications,
often without being called AI because once something becomes useful 
enough and common enough it's not labeled AI anymore."""  

docs=[Document(page_content=text_data,metadata={"source":"ABC","documentID":"123"})]
docs

[Document(metadata={'source': 'ABC', 'documentID': '123'}, page_content='Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]\n\nHigh-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); \ngenerative and creative tools (e.g., language models and AI art); \nand superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: \n"A lot

In [ ]:
# step2: chunking the document into smaller pieces

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks=splitter.split_documents(docs)
chunks

[Document(metadata={'source': 'ABC', 'documentID': '123'}, page_content='Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]'),
 Document(metadata={'source': 'ABC', 'documentID': '123'}, page_content='High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); \ngenerative and creative tools (e.g., language models and AI art); \nand superhuman play and analysis in strategy games (e.g., chess

In [ ]:
## creating embeddings for the chunks

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001"
)

# chunk_embeddings = embeddings.embed_documents([doc.page_content for doc in chunks])
# len(chunk_embeddings), len(chunk_embeddings[0])

In [ ]:
# create and store embeddings in vector store 

In [8]:
from langchain_chroma import Chroma  # New import

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="my_collection"
)

### **step 5 : semantic search**

In [9]:
vectorstore.similarity_search("what is AI?", k=2)

[Document(id='1ba8a02a-6e3d-4de3-b96e-666bdbea7800', metadata={'documentID': '123', 'source': 'ABC'}, page_content='Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]'),
 Document(id='56e9db0c-2b2e-4edb-bfa3-b17bd036e3e9', metadata={'documentID': '123', 'source': 'ABC'}, page_content='High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); \ngenerative and creative tools (e.g., language

### **Talk to LLm**

In [10]:
context = vectorstore.similarity_search("what is AI?", k=2)

In [13]:
response=llm.invoke(f"what is AI? u can answer using the following context: {context}")
answer_text = response.content[0]['text']
print(answer_text)

Based on the provided documents, **Artificial Intelligence (AI)** is defined as the capability of computational systems to perform tasks typically associated with human intelligence. These tasks include:

*   **Learning and Reasoning:** Developing methods for machines to process information.
*   **Problem-Solving and Decision-Making:** Taking actions to maximize the chances of achieving defined goals.
*   **Perception:** Enabling machines to perceive their environment.

AI is a field of research within computer science that develops software and methods to enable these capabilities. 

**Common applications of AI include:**
*   **Search and Recommendations:** Web search engines (Google Search) and recommendation systems (YouTube, Amazon, Netflix).
*   **Virtual Assistants:** Tools like Siri, Alexa, and Google Assistant.
*   **Autonomous Vehicles:** Self-driving technology such as Waymo.
*   **Generative Tools:** Language models and AI art.
*   **Gaming:** Superhuman play and analysis in